In [ ]:
#!import config/Settings.cs

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

In [ ]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

builder.WithAzureOpenAIChatCompletionService(model, azureEndpoint, apiKey, serviceId: orgId);

var kernel = builder.Build();

In [ ]:
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var aiRequestSettings = new OpenAIRequestSettings 
{
    MaxTokens = 500,
    Temperature = 0.2,
    TopP = 0.0,
    ServiceId = orgId
};

In [ ]:
var input = "I want to find top-10 books about world history";

In [ ]:
// Chat with the Bot
// Create a simple prompt for the semantic function

string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 25 words or less: ";

var getShortIntentFunction  = kernel.CreateSemanticFunction(skPrompt, aiRequestSettings, "Chat");

var intentResult = await kernel.RunAsync(input, getShortIntentFunction);

Console.WriteLine(intentResult);

In [ ]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

In [ ]:
// Chat with the Bot

aiRequestSettings = new OpenAIRequestSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5,
    ServiceId = orgId
};

var chatFunction = kernel.CreateSemanticFunction(skPrompt, aiRequestSettings);
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

var userInput = "Hi, I'm looking for suggestions of best sellers about world history";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

In [ ]:
// Update the history with the output and set this as the new input value for the next request

history += $"\nUser: {userInput}\nSuggestions: {bot_answer.GetValue<string>()}\n";
context.Variables.Update(history);

Console.WriteLine(context);

In [ ]:
// Chatting more)

Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nSuggestions: {answer.GetValue<string>()}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

In [ ]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

In [ ]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

In [ ]:
await Chat("Which topic from the ones you listed do you think the most popular?");

In [ ]:
Console.WriteLine(context.Variables["history"]);

In [ ]:
Console.WriteLine(context.Variables["userInput"]);